In [ ]:
#install yahoo finance to get data
!pip install yfinance

Data Design and Modeling Functions

In [ ]:
#import frameworks
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as LR
from sklearn.tree import DecisionTreeRegressor as DT
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_percentage_error as MAPE

#define function to create dynamical dataset
def dataset_design(data,f):
  date=data.index[3:-1]
  f1=data['Open'].values[3:-1]
  f2=data['Open'].values[2:-2]
  f3=data['Open'].values[1:-3]
  f4=data['High'].values[2:-2]
  f5=data['High'].values[1:-3]
  f6=data['Low'].values[2:-2]
  f7=data['Low'].values[1:-3]
  f8=data['Close'].values[2:-2]
  f9=data['Close'].values[1:-3]
  f10=data[f].values[:-4]
  f11=data['Volume'].values[2:-2]
  f12=data['Volume'].values[1:-3]
  target=data[f].values[3:-1]

  s={'open':f1,'open-1':f2,'open-2':f3,'high-1':f4,'high-2':f5,'low-1':f6,'low-2':f7,'close-1':f8,'close-2':f9,f+'-3':f10,'volume-1':f11,'volume-2':f12,f:target}
  data=pd.DataFrame(s,index=date,columns=s.keys())
  return data

#define model function to use autoregression
def model(data,f,pred_list,alghorithm):
  alg={'LR':LR(),'DT':DT()}
  if alghorithm in alg.keys():
    x=data.drop(f,axis=1)
    x=x.to_numpy()
    y=data[f].values
    x_train,x_test,y_train,y_test=tts(x,y,test_size=0.3,random_state=42)
    reg=alg[alghorithm]
    reg.fit(x_train,y_train)
    y_pred=reg.predict(x_test)
    y_pred_train=reg.predict(x_train)
    mape_test=(1-MAPE(y_pred,y_test))*100
    mape_train=(1-MAPE(y_pred_train,y_train))*100
    mse=MSE(y_pred,y_test)/len(x_test)
    pred=np.array(pred_list)
    pred=pred.reshape(1,-1)
    predict=float(reg.predict(pred))
    lines=[predict-mse,predict,predict+mse]

  else :
    x=data.drop(f,axis=1)
    x=x.to_numpy()
    y=data[f].values
    x_train,x_test,y_train,y_test=tts(x,y,test_size=0.3,random_state=42)
    reg=LR()
    reg.fit(x_train,y_train)
    y_pred=reg.predict(x_test)
    y_pred_train=reg.predict(x_train)
    mape_test=(1-MAPE(y_pred,y_test))*100
    mape_train=(1-MAPE(y_pred_train,y_train))*100
    mse=MSE(y_pred,y_test)/len(x_test)
    pred=np.array(pred_list)
    pred=pred.reshape(1,-1)
    predict=float(reg.predict(pred))
    lines=[predict-mse,predict,predict+mse]

  return lines,mape_train,mape_test

#deifine function to predict least candle
def predict_list(data,f):
  pred_list=[data['Open'].values[-1],
    data['Open'].values[-2],
    data['Open'].values[-3],
    data['High'].values[-2],
    data['High'].values[-3],
    data['Low'].values[-2],
    data['Low'].values[-3],
    data['Close'].values[-2],
    data['Close'].values[-3],
    data[f].values[-4],
    data['Volume'].values[-2],
    data['Volume'].values[-3] ]
  return pred_list

#define function to get predict functions and models accuracy
def index_predict_info(data,alghorithm):
  #Close
  pred_list1=predict_list(data,'Close')
  data1=dataset_design(data,'Close')
  close_pred,mape_train1,mape_test1=model(data1,'Close',pred_list1,alghorithm)
  close_pred.append(mape_train1)
  close_pred.append(mape_test1)
  #High
  pred_list2=predict_list(data,'High')
  data2=dataset_design(data,'High')
  high_pred,mape_train2,mape_test2=model(data2,'High',pred_list2,alghorithm)
  high_pred.append(mape_train2)
  high_pred.append(mape_test2)
  #Low
  pred_list3=predict_list(data,'Low')
  data3=dataset_design(data,'Low')
  low_pred,mape_train3,mape_test3=model(data3,'Low',pred_list3,alghorithm)
  low_pred.append(mape_train3)
  low_pred.append(mape_test3)

  s={'Low':low_pred,'High':high_pred,'Close':close_pred}
  index=['Perdicted price - mse','Perdicted price','Perdicted price + price','Train Accuracy','Test Accuracy']
  df=pd.DataFrame(s,index=index,columns=s.keys())
  return df

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


BTC

In [ ]:
#daily canddle
symbol='BTC-USD'
data=yf.Ticker(symbol)
Data=data.history(period='7y',interval='1d')
df_BTC=index_predict_info(Data,'LR')
df_BTC

<ipython-input-2-31fe4af713a9>:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predict=float(reg.predict(pred))
<ipython-input-2-31fe4af713a9>:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predict=float(reg.predict(pred))
<ipython-input-2-31fe4af713a9>:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predict=float(reg.predict(pred))


,Low,High,Close
Perdicted price - mse,50457.142710,51874.120733,50570.614177
Perdicted price,50967.867080,52505.986943,51711.872173
Perdicted price + price,51478.591450,53137.853153,52853.130168
Train Accuracy,97.896090,98.287679,97.382904
Test Accuracy,97.986394,98.108410,97.362095


In [ ]:
#60min canddle
symbol='BTC-USD'
data=yf.Ticker(symbol)
Data=data.history(period='2y',interval='60m')
df_BTC=index_predict_info(Data,'LR')
df_BTC

<ipython-input-2-31fe4af713a9>:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predict=float(reg.predict(pred))
<ipython-input-2-31fe4af713a9>:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predict=float(reg.predict(pred))
<ipython-input-2-31fe4af713a9>:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predict=float(reg.predict(pred))


,Low,High,Close
Perdicted price - mse,54423.335650,54851.840806,54670.857630
Perdicted price,54425.976471,54854.173772,54675.940985
Perdicted price + price,54428.617292,54856.506737,54681.024339
Train Accuracy,99.785617,99.787455,99.683346
Test Accuracy,99.779914,99.785691,99.677925


In [ ]:
Data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2022-02-26 22:00:00+00:00,39438.851562,39451.820312,39184.023438,39184.023438,0,0.0,0.0
2022-02-26 23:00:00+00:00,39164.679688,39164.679688,38982.511719,39107.500000,0,0.0,0.0
2022-02-27 00:00:00+00:00,39098.699219,39216.593750,38811.968750,38877.792969,0,0.0,0.0
2022-02-27 01:00:00+00:00,38831.093750,38851.542969,38431.078125,38442.703125,292761600,0.0,0.0
2022-02-27 02:00:00+00:00,38393.292969,38578.351562,38293.500000,38578.351562,244993024,0.0,0.0
...,...,...,...,...,...,...,...
2024-02-26 18:00:00+00:00,53506.113281,53648.117188,53336.503906,53572.492188,1469149184,0.0,0.0
2024-02-26 19:00:00+00:00,53603.753906,54524.589844,53489.906250,54454.117188,3349655552,0.0,0.0
2024-02-26 20:00:00+00:00,54523.378906,54938.175781,54310.406250,54454.527344,1764153344,0.0,0.0
